# Earthquake in Turkey

 - Koeri dataset


## Prep Libraries

In [ ]:
# Load up libraries
import pandas as pd
import folium
import datetime as dt
import sqlite3

print('Pandas: ', pd.__version__)
print('Folium: ', folium.__version__)

## Historic Data Loading from the CSV file


In [ ]:
# Load u history data from csv file
histDF = pd.read_csv("koeri.boun.edu.tr-lasteq-autosol_from_2023-01-16_tab.csv", sep='\t')
histDF

In [ ]:
# Treatment of some data
histDF['depth(km)'] = histDF['depth(km)'].replace('-', 0).astype("float")
histDF['depth(km)'] = histDF['depth(km)'].replace('E', 0).astype("float")
# Naming the columns
histDF.columns = ["origintimeutc", "magnitude", "magType", "latitude", "longitude", "depthKm", "region", "measMethod", "updTime", "attribute"]
# Reprting the depthKm column
if len(histDF['origintimeutc']) == len(histDF['depthKm']):
    print("All look good, proceed!")
else:
    print("The depthKm column has some wrong data type, please investigate!")

In [ ]:
# Sort the DF to be the format of: Older points on top prior to inserting into database
histDF1 = histDF.sort_values(by='origintimeutc', ascending=True)
histDF1.reset_index(drop=True, inplace=True)
histDF1

## Save Dataframe to a SQLite3 database

In [ ]:
# SAVE RESULTANT DF TO SQLITE3 DATABASE

# Create sqlite database and cursor
conn = sqlite3.connect('test.db')
c = conn.cursor()
# Create the table of quaketk
c.execute("""CREATE TABLE IF NOT EXISTS quaketk_koeri (
            origintimeutc text,
            magnitude real,
            magtype text,
            latitude real,
            longitude real,
            depthkm real,
            region text,
            measmethod text,
            updtime text,
            attribute text
            )""")
conn.commit()

# test
test = conn.execute('SELECT * from quaketk_koeri')
names = [description[0] for description in test.description]
print(names)

In [ ]:
# Save the dataframe to database
histDF.columns = ['origintimeutc', 'magnitude', 'magtype', 'latitude', 'longitude', 'depthkm', 'region', 'measmethod', 'updtime', 'attribute']
histDF.to_sql('quaketk_koeri', conn, if_exists='append', index=False)

# verify if the dataframe has been saved to sqlite DB or not, please uncomment the following command
# conn.execute('SELECT * from quaketk').fetchall()

## Retrieve the whole dataset and Display

In [ ]:
# Setup pandas display template

pd.set_option('display.width', 800)
pd.set_option('display.max_columns', 10)

In [ ]:
# Read the Dataset from the database
import sqlite3
# Create a connection to the databse
conn = sqlite3.connect('test.db')

# Read out the whole dataset as dataframe
df = pd.read_sql_query("SELECT * FROM quaketk_koeri", conn)
df

In [ ]:
print('Time Range of the Dataset is between:', df['origintimeutc'].min(), 'and', df['origintimeutc'].max())

In [ ]:
# Re-treatment of the dataset
df['origintimeutc'] = df['origintimeutc'].apply(lambda x: dt.datetime.strptime(x,'%Y/%m/%d %H:%M:%S') if type(x)==str else pd.NaT)
df['magnitude'] = df['magnitude'].astype('float')
df['magtype'] = df['magtype'].astype('string')

df['latitude'] = df['latitude'].astype(str).map(lambda x: x.rstrip('° N').rstrip('° S'))
df['longitude'] = df['longitude'].astype(str).map(lambda x: x.rstrip('° E').rstrip('° W'))
df['depthkm'] = df['depthkm'].replace('-', 0).astype("float")

df['region'] = df['region'].astype('string')
df['measmethod'] = df['measmethod'].astype('string')
df['updtime'] = df['updtime'].apply(lambda x: dt.datetime.strptime(x,'%Y/%m/%d %H:%M:%S') if type(x)==str else pd.NaT)
df['attribute'] = df['attribute'].astype('string')

# Adjust datetime from UTC (GMT) to Turkey timezone (GMT+3)
df['eventtime'] = df['origintimeutc'] + pd.DateOffset(hours=3)
df['updtime'] = df['updtime'] + pd.DateOffset(hours=3)

# Create new columns for date and time
df['date'] = pd.to_datetime(df['eventtime']).dt.date
df['time'] = pd.to_datetime(df['eventtime']).dt.time

# take a look
df.head()


In [ ]:
df.columns

In [ ]:
# Subset the df and rename the columns
df2 = df[['date', 'time', 'latitude', 'longitude', 'depthkm', 'magnitude', 'magtype', 'region', 'measmethod', 'eventtime', 'updtime', 'attribute']]
df2.head()
# print the earthquake with magnitude >=4
print(df2[df2['magnitude'].astype('float') >= 4])

In [ ]:
# Define our functions
def addCircles(df, map):
    for x, y, z, loc, date, time in zip(df['latitude'], df['longitude'], df['magnitude'], df['region'], df['date'], df['time']):
        popUp = f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Magnitude: {z} ML&nbsp;</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Location: {loc}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Date: {date}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Time: {time}</strong></span></p>"
        iframe = folium.IFrame(popUp, width=250, height=150)
        popup = folium.Popup(iframe, max_width=450)
        folium.CircleMarker(location=(x, y), radius=float(z) * 4, weight=2, opacity=1, popup=popup,
                            color="red", fill_color="red", fill_opacity=0.1).add_to(map)

In [ ]:
# Setup initial map

magThreshold = 5

earthquakeMap = folium.Map(location=[39.16, 35.66], zoom_start=6, tiles=None)

# Load up the polygon (in GeoJson format) around turkey
# Original URL was - https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json
folium.GeoJson('../data/PB2002_boundaries.json', name="Tectonic Boundaries").add_to(earthquakeMap)
#folium.GeoJson('data/geoboundaries-TUR-ADM1_simplified.geojson', name="Turkey Provinces").add_to(earthquakeMap)

# Add tiles (The first tile will be the default)
folium.TileLayer('Stamen Terrain', name='Terrian').add_to(earthquakeMap)
folium.TileLayer('openstreetmap', name='Open Street').add_to(earthquakeMap)
folium.TileLayer('Stamen Toner', name='Toner').add_to(earthquakeMap)
folium.TileLayer('Stamen Water Color', name='Water Color').add_to(earthquakeMap)
folium.TileLayer('cartodbdark_matter', name='Dark Matter').add_to(earthquakeMap)
folium.LayerControl().add_to(earthquakeMap)

# add Circles (sizing per magnitude)
earthquakeDF = df2[df2['magnitude'].astype('float') >= magThreshold]
addCircles(earthquakeDF, earthquakeMap)

# Take a look at the map
earthquakeMap


## The End